In [29]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv("data/bpost.csv").dropna().drop('Forcast', axis=1)
df

,Datum,Actual
0,03/08/2020,466.0
1,04/08/2020,633.0
2,05/08/2020,685.0
3,06/08/2020,601.0
4,07/08/2020,607.0
...,...,...
124,25/01/2021,856.0
125,26/01/2021,853.0
126,27/01/2021,918.0
127,28/01/2021,803.0


In [14]:
df.Datum.nunique()

129

In [34]:
from avatar import WranglingLanguage, Expander
from avatar.transformations import TimeFeatures
from avatar.settings import Settings
from avatar.filter import UniqueFilter

Settings.verbose = True

In [35]:
language = WranglingLanguage([TimeFeatures])
expander = Expander(language)
expanded = UniqueFilter().select(expander.expand(df, exclude=["Actual"]))
expanded = expanded.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
expanded.head()

100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


,Actual,TimeFeatures()(Datum)_w,TimeFeatures()(Datum)_-d,TimeFeatures()(Datum)_-m,TimeFeatures()(Datum)_-y,TimeFeatures()(Datum)_Y,TimeFeatures()(Datum)_-j,TimeFeatures()(Datum)_U,TimeFeatures()(Datum)_W
0,466.0,0,8,3,20,2020,68,10,9
1,633.0,3,8,4,20,2020,99,14,14
2,685.0,5,8,5,20,2020,129,18,18
3,601.0,1,8,6,20,2020,160,23,23
4,607.0,3,8,7,20,2020,190,27,27


In [36]:
from avatar.supervised import AnnealingTournament, LegacyTournament

In [37]:
legacy = LegacyTournament(games=400)
legacy.initialise(expanded, target="Actual")
legacy.play()

100%|██████████| 400/400 [00:08<00:00, 45.71it/s]


In [38]:
legacy.results

{'TimeFeatures()(Datum)_Y': 0.05750133809330579,
 'TimeFeatures()(Datum)_-d': 0.2357857294334564,
 'TimeFeatures()(Datum)_U': 0.13489988915144133,
 'TimeFeatures()(Datum)_w': 0.057683393822469835,
 'TimeFeatures()(Datum)_-m': 0.07145651335037817,
 'TimeFeatures()(Datum)_W': 0.13029075065731285,
 'TimeFeatures()(Datum)_-j': 0.2749407581146985,
 'TimeFeatures()(Datum)_-y': 0.03744162737693718}

In [39]:
from avatar.supervised import Game, AveragePool

anneal = AnnealingTournament(
    Game(rounds=5),
    AveragePool(),
    games=400,
    exploration=0.5,
    teamsize=4
)
anneal.initialise(expanded, target="Actual")
anneal.play()

4: 100%|██████████| 400/400 [00:43<00:00,  8.74it/s] 

In [40]:
anneal.results

{'TimeFeatures()(Datum)_Y': 0.0014398469329329887,
 'TimeFeatures()(Datum)_-d': 0.3705984925865423,
 'TimeFeatures()(Datum)_U': 0.04261023743734611,
 'TimeFeatures()(Datum)_w': 0.016491520015528314,
 'TimeFeatures()(Datum)_-m': 0.014251828012883532,
 'TimeFeatures()(Datum)_W': 0.048864638700235996,
 'TimeFeatures()(Datum)_-j': 0.5049854876884274,
 'TimeFeatures()(Datum)_-y': 0.0007579486261033056}

In [41]:
from avatar.evaluate import RankingEvaluator

In [42]:
estimator = RandomForestRegressor(max_depth=4)
evaluator = RankingEvaluator(estimator, max_features=10)

In [43]:
evaluator.fit(expanded, target="Actual", ranking=legacy.results)
evaluator.scores


4: 100%|██████████| 400/400 [00:46<00:00,  8.52it/s]

100%|██████████| 9/9 [00:12<00:00,  1.38s/it]


{1: {'scores': [-0.07068439655836153,
   -0.14504503081765457,
   -0.3003476368658234,
   0.019665460550896397,
   -0.39522748197645885],
  'features': ['TimeFeatures()(Datum)_-j']},
 2: {'scores': [0.3366916436594877,
   0.4425045182502322,
   0.14841320268516728,
   0.18318044375791936,
   0.25266156528246514],
  'features': ['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']},
 3: {'scores': [0.28919582306445274,
   0.16908238496290043,
   0.10251187220457085,
   0.1789785948229407,
   0.08158350049205851],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_U']},
 4: {'scores': [-0.023319055879807225,
   0.14606985643169673,
   0.26709867778813123,
   0.12216947503454459,
   0.011614565524274357],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_U',
   'TimeFeatures()(Datum)_W']},
 5: {'scores': [0.39117891786537634,
   0.12120397452781424,
   0.4452189871372817,
   -0.0073463320

In [44]:
for k, v in _.items():
    print(k, np.mean(v['scores']))
    print(v['features'])
    print()

1 -0.1783278171334804
['TimeFeatures()(Datum)_-j']

2 0.2726902747270543
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']

3 0.16427043510938463
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U']

4 0.10472670377976792
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W']

5 0.22225443009821166
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_-m']

6 0.20308969289563694
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_-m', 'TimeFeatures()(Datum)_w']

7 0.3479778748273305
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_-m', 'TimeFeatures()(Datum)_w', 'TimeFeatures()(Datum)_Y']

8 0.08732406005899032
['TimeFeatures()(Datum)_-j', 'TimeFeatur

In [45]:
evaluator.fit(expanded, target="Actual", ranking=anneal.results)
evaluator.scores

100%|██████████| 9/9 [00:08<00:00,  1.11it/s]


{1: {'scores': [-0.2901200890977193,
   0.053127475547072,
   0.10613613380377293,
   -0.33770925861357765,
   -0.10512662609761274],
  'features': ['TimeFeatures()(Datum)_-j']},
 2: {'scores': [0.4452860349580404,
   0.4698956138958191,
   0.01808151428836635,
   0.1331927032664944,
   0.45003046142713965],
  'features': ['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']},
 3: {'scores': [0.11981540304375574,
   0.16219700737533527,
   0.09320744055284398,
   0.27292681304341126,
   0.4203465183536606],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_W']},
 4: {'scores': [0.35331377260010643,
   0.08255950575635014,
   0.12829466521891877,
   0.1921441812525388,
   0.1553398456372363],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_W',
   'TimeFeatures()(Datum)_U']},
 5: {'scores': [0.1696323669887528,
   -0.010041350286093053,
   0.12267417737677311,
   0.06643417867234795,


In [46]:
for k, v in _.items():
    print(k, np.mean(v['scores']))
    print(v['features'])
    print()

1 -0.11473847289161294
['TimeFeatures()(Datum)_-j']

2 0.30329726556717196
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']

3 0.21369863647380133
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W']

4 0.1823303940930301
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_U']

5 0.10143800108582654
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_w']

6 0.1564646250747166
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_w', 'TimeFeatures()(Datum)_-m']

7 0.2569046791759416
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_w', 'TimeFeatures()(Datum)_-m', 'TimeFeatures()(Datum)_Y']

8 0.24541300274276617
['TimeFeatures()(Datum)_-j', 'TimeFeature